In [7]:
import tkinter as tk
from tkinter import messagebox
import time
import sched
from plyer import notification
import json

# Initialize the scheduler
scheduler = sched.scheduler(time.time, time.sleep)

# Create a list to store remainders
remainders = []

def set_remainder(title, message, delay):
    scheduler.enter(delay, 1, show_notification, (title, message))
    remainders.append({
        "title": title,
        "message": message,
        "delay": delay,
        "time_set": time.time()
    })
    save_remainders()

def show_notification(title, message):
    notification.notify(
        title=title,
        message=message,
        timeout=10  # Notification display duration in seconds
    )

def save_remainders():
    with open('remainders.json', 'w') as file:
        json.dump(remainders, file)

def load_remainders():
    global remainders
    try:
        with open('remainders.json', 'r') as file:
            remainders = json.load(file)
            for rem in remainders:
                delay = rem['delay'] - (time.time() - rem['time_set'])
                if delay > 0:
                    scheduler.enter(delay, 1, show_notification, (rem['title'], rem['message']))
    except FileNotFoundError:
        remainders = []

def set_remainder_gui():
    title = title_entry.get()
    message = message_entry.get()
    delay = int(delay_entry.get())

    set_remainder(title, message, delay)
    messagebox.showinfo("Success", "Remainder set successfully!")

app = tk.Tk()
app.title("Remainder Application")

tk.Label(app, text="Title:").grid(row=0)
tk.Label(app, text="Message:").grid(row=1)
tk.Label(app, text="Delay (seconds):").grid(row=2)

title_entry = tk.Entry(app)
message_entry = tk.Entry(app)
delay_entry = tk.Entry(app)

title_entry.grid(row=0, column=1)
message_entry.grid(row=1, column=1)
delay_entry.grid(row=2, column=1)

tk.Button(app, text="Set Remainder", command=set_remainder_gui).grid(row=3, columnspan=2)

load_remainders()
app.mainloop()

# Run the scheduler
scheduler.run()


In [8]:
pip install pyqt5



SyntaxError: invalid syntax (4203634802.py, line 1)

In [9]:
import sys
import time
import sched
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel, QLineEdit, QPushButton, QMessageBox
from PyQt5.QtCore import QTimer
from plyer import notification
import json

# Initialize the scheduler
scheduler = sched.scheduler(time.time, time.sleep)

# Create a list to store remainders
remainders = []

def set_remainder(title, message, delay):
    scheduler.enter(delay, 1, show_notification, (title, message))
    remainders.append({
        "title": title,
        "message": message,
        "delay": delay,
        "time_set": time.time()
    })
    save_remainders()

def show_notification(title, message):
    notification.notify(
        title=title,
        message=message,
        timeout=10  # Notification display duration in seconds
    )

def save_remainders():
    with open('remainders.json', 'w') as file:
        json.dump(remainders, file)

def load_remainders():
    global remainders
    try:
        with open('remainders.json', 'r') as file:
            remainders = json.load(file)
            for rem in remainders:
                delay = rem['delay'] - (time.time() - rem['time_set'])
                if delay > 0:
                    scheduler.enter(delay, 1, show_notification, (rem['title'], rem['message']))
    except FileNotFoundError:
        remainders = []

class RemainderApp(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
    
    def initUI(self):
        self.setWindowTitle('Remainder Application')
        
        layout = QVBoxLayout()
        
        self.title_label = QLabel('Title:')
        self.title_input = QLineEdit()
        
        self.message_label = QLabel('Message:')
        self.message_input = QLineEdit()
        
        self.delay_label = QLabel('Delay (seconds):')
        self.delay_input = QLineEdit()
        
        self.set_button = QPushButton('Set Remainder')
        self.set_button.clicked.connect(self.set_remainder_gui)
        
        layout.addWidget(self.title_label)
        layout.addWidget(self.title_input)
        layout.addWidget(self.message_label)
        layout.addWidget(self.message_input)
        layout.addWidget(self.delay_label)
        layout.addWidget(self.delay_input)
        layout.addWidget(self.set_button)
        
        self.setLayout(layout)
    
    def set_remainder_gui(self):
        title = self.title_input.text()
        message = self.message_input.text()
        delay = int(self.delay_input.text())
        
        set_remainder(title, message, delay)
        QMessageBox.information(self, 'Success', 'Remainder set successfully!')

app = QApplication(sys.argv)
remainder_app = RemainderApp()
remainder_app.show()

load_remainders()

# Timer to keep the scheduler running
timer = QTimer()
timer.timeout.connect(lambda: scheduler.run(blocking=False))
timer.start(1000)

sys.exit(app.exec_())


ValueError: invalid literal for int() with base 10: '0.1'

ValueError: invalid literal for int() with base 10: '0.1'

SystemExit: 0

C:\Users\brije\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import sqlite3
import time
import sched
from plyer import notification
import tkinter as tk
from tkinter import messagebox

# Initialize the scheduler
scheduler = sched.scheduler(time.time, time.sleep)

# Initialize SQLite
conn = sqlite3.connect('remainders.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS remainders
             (title TEXT, message TEXT, delay INTEGER, time_set REAL)''')
conn.commit()

def set_remainder(title, message, delay):
    scheduler.enter(delay, 1, show_notification, (title, message))
    c.execute("INSERT INTO remainders (title, message, delay, time_set) VALUES (?, ?, ?, ?)", 
              (title, message, delay, time.time()))
    conn.commit()

def show_notification(title, message):
    notification.notify(
        title=title,
        message=message,
        timeout=10  # Notification display duration in seconds
    )

def load_remainders():
    c.execute("SELECT * FROM remainders")
    rows = c.fetchall()
    for row in rows:
        title, message, delay, time_set = row
        delay = delay - (time.time() - time_set)
        if delay > 0:
            scheduler.enter(delay, 1, show_notification, (title, message))

def set_remainder_gui():
    title = title_entry.get()
    message = message_entry.get()
    delay = int(delay_entry.get())

    set_remainder(title, message, delay)
    messagebox.showinfo("Success", "Remainder set successfully!")

app = tk.Tk()
app.title("Remainder Application")

tk.Label(app, text="Title:").grid(row=0)
tk.Label(app, text="Message:").grid(row=1)
tk.Label(app, text="Delay (seconds):").grid(row=2)

title_entry = tk.Entry(app)
message_entry = tk.Entry(app)
delay_entry = tk.Entry(app)

title_entry.grid(row=0, column=1)
message_entry.grid(row=1, column=1)
delay_entry.grid(row=2, column=1)

tk.Button(app, text="Set Remainder", command=set_remainder_gui).grid(row=3, columnspan=2)

load_remainders()
app.mainloop()

# Run the scheduler
scheduler.run()

# Close SQLite connection
conn.close()


In [ ]:
def list_remainders():
    listbox.delete(0, tk.END)
    for rem in remainders:
        listbox.insert(tk.END, f"Title: {rem['title']}, Message: {rem['message']}, Delay: {rem['delay']}")

app = tk.Tk()
app.title("Remainder Application")

tk.Label(app, text="Title:").grid(row=0)
tk.Label(app, text="Message:").grid(row=1)
tk.Label(app, text="Delay (seconds):").grid(row=2)

title_entry = tk.Entry(app)
message_entry = tk.Entry(app)
delay_entry = tk.Entry(app)

title_entry.grid(row=0, column=1)
message_entry.grid(row=1, column=1)
delay_entry.grid(row=2, column=1)

tk.Button(app, text="Set Remainder", command=set_remainder_gui).grid(row=3, columnspan=2)
listbox = tk.Listbox(app)
listbox.grid(row=4, columnspan=2)

tk.Button(app, text="List Remainders", command=list_remainders).grid(row=5, columnspan=2)

load_remainders()
app.mainloop()


In [ ]:
def delete_remainder():
    selected = listbox.curselection()
    if selected:
        index = selected[0]
        rem = remainders[index]
        c.execute("DELETE FROM remainders WHERE title=? AND message=? AND delay=? AND time_set=?", 
                  (rem['title'], rem['message'], rem['delay'], rem['time_set']))
        conn.commit()
        remainders.pop(index)
        list_remainders()

tk.Button(app, text="Delete Remainder", command.delete_remainder).grid(row=6, columnspan=2)
